In [1]:
import tensorflow as tf
import numpy as np

In [23]:
# Some toy data to try
user_ids = [1, 2, 3, 4, 5]
item_ids = [101, 102, 103, 104, 105, 106, 107]
# We represent each user with the items he interactived with
user_items_links = [
    (1, [101, 103, 105]), 
    (2, [102, 104]), 
    (3, [103, 107]), 
    (4, [102, 106]), 
    (5, [101, 105, 107])
]
training_data = np.array(
    [[1, 0, 1, 0, 1, 0, 0],
     [1, 0, 1, 0, 1, 0, 0],
     [1, 0, 1, 0, 1, 0, 0],
     [0, 1, 0, 1, 0, 0, 0],
     [0, 1, 0, 1, 0, 0, 0],
     [0, 0, 1, 0, 1, 0, 0],
     [0, 0, 1, 0, 1, 0, 0],
     [0, 1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1, 0],
     [1, 0, 0, 0, 1, 0, 1],
     [1, 0, 0, 0, 1, 0, 1],
     [1, 0, 0, 0, 1, 0, 1]], dtype=float
)
training_data = training_data / np.reshape(np.sum(training_data, axis=1), (training_data.shape[0], 1))
training_labels = np.array([0, 2, 4, 1, 3, 2, 6, 1, 5, 0, 4, 6])

In [48]:
"""
Set up the neural network, the overall workflow is:
    
    fetch_embeddings -> avg -> fc -> relu -> softmax loss
"""
num_items = len(item_ids)
embedding_dim = 128
n_hidden_1 = 256
n_hidden_2 = 128
n_classes = 7
batch_size = 12

graph = tf.Graph()

with graph.as_default():
    
    x = tf.placeholder(tf.float32, shape=[batch_size, num_items])
    y = tf.placeholder(tf.int32, shape=[batch_size])
    
    embeddings = tf.Variable(
        tf.random_uniform([num_items, embedding_dim], -1.0, 1.0))
    
    embed = tf.matmul(x, embeddings)
    
    W1 = tf.Variable(tf.random_normal([embedding_dim, n_hidden_1]))
    b1 = tf.Variable(tf.random_normal([n_hidden_1]))
    
    layer_1 = tf.add(tf.matmul(embed, W1), b1)
    layer_1 = tf.nn.relu(layer_1)
    
    W2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
    b2 = tf.Variable(tf.random_normal([n_hidden_2]))
    
    layer_2 = tf.add(tf.matmul(layer_1, W2), b2)
    layer_2 = tf.nn.relu(layer_2)
    
    # TODO: make sure this step is correct
    out = tf.matmul(layer_2, tf.transpose(embeddings))
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=out, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
    
    init = tf.initialize_all_variables()

with tf.Session(graph=graph) as session:
    init.run()
    print("Initialized")
    
    for i in range(10):
        _, loss_val = session.run([optimizer, loss], feed_dict={
            x: training_data,
            y: training_labels,
        })
        
        print loss_val
    
    user_embeddings = training_data.dot(embeddings.eval())
    user_items = user_embeddings.dot(embeddings.eval().T)
    print np.argmax(user_items, axis=1)
        

Initialized
466.086
203.5
124.884
103.161
81.2097
108.516
101.81
90.356
73.1478
64.3048
[0 0 0 1 1 2 2 1 1 0 0 0]
